## Projet Recapitulatif- Problématique en Camdbogde
    Problématique :
Identifier les attractions touristiques les plus populaires et les caractéristiques des hôtels les mieux notés dans deux destinations clés du Cambodge – Siem Reap (Angkor Wat) et Sihanoukville – afin de fournir des recommandations pour la conception d’un nouvel hôtel adapté aux attentes des visiteurs.

-Objectifs du projet

 Analyser les hôtels existants à Siem Reap et Sihanoukville : prix, notes, services proposés.

 Identifier les attractions les plus visitées et les mieux notées autour de ces deux villes.

 Comparer les deux destinations pour voir les différences de profil touristique (culture vs plage).

 Proposer des recommandations pour optimiser l’offre hôtelière (prix cible, services prioritaires).

In [3]:
!pip install requests beautifulsoup4 selenium pandas numpy sqlalchemy mysql-connector-python flask
!pip install jupyter notebook matplotlib seaborn plotly


  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached beautifulsoup4-4.13.5-py3-none-any.whl.metadata (3.8 kB)
  Using cached selenium-4.35.0-py3-none-any.whl.metadata (7.4 kB)
  Using cached pandas-2.3.2-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached sqlalchemy-2.0.43-cp311-cp311-win_amd64.whl.metadata (9.8 kB)
  Using cached mysql_connector_python-9.4.0-cp311-cp311-win_amd64.whl.metadata (7.5 kB)
  Using cached flask-3.1.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached charset_normalizer-3.4.3-cp311-cp311-win_amd64.whl.metadata (37 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached soupsieve-2.8-py3-none-any.whl.metadata (4.6 kB)
  Using cached trio-0.30.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached typing_exten

In [26]:
pip install selenium webdriver-manager


Note: you may need to restart the kernel to use updated packages.


In [48]:
url="https://www.agoda.com/fr-fr/city/siem-reap-kh.html"

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import csv
import time

# --- Configuration du CSV ---
fichier_csv = "hotels_cambodge.csv"
with open(fichier_csv, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Ville", "Nom", "Adresse", "Étoiles", "Note", "Avis", "Prix"])

# --- Config du navigateur ---
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/139.0.7258.155 Safari/537.36"
)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# --- Liste des villes ---
villes = {
    "Siem Reap": "https://www.agoda.com/fr-fr/city/siem-reap-kh.html",
    "Sihanoukville": "https://www.agoda.com/fr-fr/city/sihanoukville-kh.html",
    "Phnom Penh": "https://www.agoda.com/fr-fr/city/phnom-penh-kh.html",
    "Battambang": "https://www.agoda.com/fr-fr/city/battambang-kh.html",
    "Kampot": "https://www.agoda.com/fr-fr/city/kampot-kh.html",
    "Kampong Cham": "https://www.agoda.com/fr-fr/city/kampong-cham-kh.html",
    "Kampong Thom": "https://www.agoda.com/fr-fr/city/kampong-thom-kh.html",
    "Koh Kong": "https://www.agoda.com/fr-fr/city/koh-kong-kh.html",
    "Kratie": "https://www.agoda.com/fr-fr/city/kratie-kh.html",
    "Mondulkiri": "https://www.agoda.com/fr-fr/city/mondulkiri-kh.html",
    "Preah Vihear": "https://www.agoda.com/fr-fr/city/preah-vihear-kh.html",
    "Pursat": "https://www.agoda.com/fr-fr/city/pursat-kh.html",
    "Ratanakiri": "https://www.agoda.com/fr-fr/city/ratanakiri-kh.html",
    "Stung Treng": "https://www.agoda.com/fr-fr/city/stung-treng-kh.html",
    "Svay Rieng": "https://www.agoda.com/fr-fr/city/svay-rieng-kh.html",
    "Takeo": "https://www.agoda.com/fr-fr/city/takeo-kh.html",
    "Tbong Khmum": "https://www.agoda.com/fr-fr/city/tbong-khmum-kh.html",
}

# --- Boucle sur les villes ---
for ville, url in villes.items():
    print(f"🌍 Scraping des hôtels pour {ville}...")
    driver.get(url)
    time.sleep(5)

    # Scroll pour charger plusieurs hôtels (Agoda charge par paquets)
    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(3):  # tu peux augmenter si tu veux charger plus
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    cartes = driver.find_elements(By.CSS_SELECTOR, "div.DatelessPropertyCard")
    print(f"🔎 {len(cartes)} hôtels trouvés dans {ville}")

    for card in cartes:
        try:
            nom = card.find_element(By.CSS_SELECTOR, "div.DatelessPropertyCard__ContentHeader").text
        except:
            nom = "N/A"

        try:
            adresse = card.find_element(By.CSS_SELECTOR, "span.DatelessPropertyCard__ContentAreaCity").text
        except:
            adresse = "N/A"

        try:
            etoiles = card.find_element(By.CSS_SELECTOR, "i[data-selenium='hotel-star-rating']").get_attribute("aria-label")
        except:
            etoiles = "N/A"

        try:
            note = card.find_element(By.CSS_SELECTOR, "span.Spanstyled__SpanStyled-sc-16tp9kb-0.kpRlPc").text
        except:
            note = "N/A"

        try:
            avis_text = card.find_element(By.XPATH, ".//p[contains(text(),'avis')]").text
            avis = avis_text.replace("\xa0", "").replace("avis", "").strip()
        except:
            avis = "N/A"

        try:
            prix = card.find_element(By.CSS_SELECTOR, "div.DatelessPropertyCard__AdditionalPriceCurrency").text
        except:
            prix = "N/A"

        with open(fichier_csv, mode="a", newline="", encoding="utf-8") as file:
            writer = csv.writer(file)
            writer.writerow([ville, nom, adresse, etoiles, note, avis, prix])

        time.sleep(0.5)

driver.quit()
print(f"✅ Données enregistrées dans {fichier_csv}")


🌍 Scraping des hôtels pour Siem Reap...
🔎 10 hôtels trouvés dans Siem Reap
🌍 Scraping des hôtels pour Sihanoukville...
🔎 10 hôtels trouvés dans Sihanoukville
🌍 Scraping des hôtels pour Phnom Penh...
🔎 10 hôtels trouvés dans Phnom Penh
🌍 Scraping des hôtels pour Battambang...
🔎 10 hôtels trouvés dans Battambang
🌍 Scraping des hôtels pour Kampot...
🔎 10 hôtels trouvés dans Kampot
🌍 Scraping des hôtels pour Kampong Cham...
🔎 10 hôtels trouvés dans Kampong Cham
🌍 Scraping des hôtels pour Kampong Thom...
🔎 4 hôtels trouvés dans Kampong Thom
🌍 Scraping des hôtels pour Koh Kong...
🔎 6 hôtels trouvés dans Koh Kong
🌍 Scraping des hôtels pour Kratie...
🔎 10 hôtels trouvés dans Kratie
🌍 Scraping des hôtels pour Mondulkiri...
🔎 0 hôtels trouvés dans Mondulkiri
🌍 Scraping des hôtels pour Preah Vihear...
🔎 4 hôtels trouvés dans Preah Vihear
🌍 Scraping des hôtels pour Pursat...
🔎 2 hôtels trouvés dans Pursat
🌍 Scraping des hôtels pour Ratanakiri...
🔎 0 hôtels trouvés dans Ratanakiri
🌍 Scraping des hô

In [41]:
import pandas as pd

df = pd.read_csv("hotels_cambodge.csv")
print(f"Nombre total d'hôtels : {len(df)}")
print(df.tail(5))  # voir les 5 dernières lignes


Nombre total d'hôtels : 126
                Ville                               Nom  \
121  Koh Rong Sanloem                  SoonNoeng Resort   
122  Koh Rong Sanloem  Paradise Villas Koh Rong Sanloem   
123  Koh Rong Sanloem           One Beach Resort By EHM   
124  Koh Rong Sanloem          Onederz Koh Rong Sanloem   
125  Koh Rong Sanloem                Tube Resort By EHM   

                                              Adresse    Étoiles  Note   Avis  \
121  Saracen Bay,Koh Rong Samloem - Voir sur la carte  4 étoiles   8.6   76.0   
122  Saracen Bay,Koh Rong Samloem - Voir sur la carte  3 étoiles   8.1  167.0   
123  Saracen Bay,Koh Rong Samloem - Voir sur la carte  4 étoiles   8.0  187.0   
124  Saracen Bay,Koh Rong Samloem - Voir sur la carte   1 étoile   8.8  455.0   
125  Saracen Bay,Koh Rong Samloem - Voir sur la carte  4 étoiles   8.3  411.0   

     Prix  
121  64 €  
122  37 €  
123  80 €  
124   NaN  
125  68 €  


In [42]:
df.head(10)

,Ville,Nom,Adresse,Étoiles,Note,Avis,Prix
0,Siem Reap,Onederz Hostel Siem Reap,"Siem Reap centre,Siem Reap - Voir sur la carte",3 étoiles,9.3,3456.0,12 €
1,Siem Reap,Pandora Suite D'Angkor,"Kouk Chak,Siem Reap - Voir sur la carte",5 étoiles,9.2,1004.0,31 €
2,Siem Reap,Elysium Suite La Residence,"Sala Kamreuk,Siem Reap - Voir sur la carte",4 étoiles,9.1,439.0,32 €
3,Siem Reap,Grand Venus La Residence,"Kouk Chak,Siem Reap - Voir sur la carte",NaN,9.0,1040.0,31 €
4,Siem Reap,Lub d Siem Reap,"Siem Reap centre,Siem Reap - Voir sur la carte",4 étoiles,9.3,3129.0,14 €
5,Siem Reap,Bopha Wat Bo Residence,"Siem Reap centre,Siem Reap - Voir sur la carte",5 étoiles,9.2,285.0,29 €
6,Siem Reap,Indra Angkor Residence,"Sala Kamreuk,Siem Reap - Voir sur la carte",4 étoiles,8.7,597.0,25 €
7,Siem Reap,Darling Pub Hostel,"Siem Reap centre,Siem Reap - Voir sur la carte",3 étoiles,8.9,894.0,19 €
8,Siem Reap,VPlus Hotel,"Siem Reap centre,Siem Reap - Voir sur la carte",3 étoiles,9.4,1468.0,10 €
9,Siem Reap,Tropical Breeze Guesthouse,"Siem Reap centre,Siem Reap - Voir sur la carte",2 étoiles,8.2,550.0,14 €


In [43]:
df.head()
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Ville    126 non-null    object 
 1   Nom      126 non-null    object 
 2   Adresse  126 non-null    object 
 3   Étoiles  107 non-null    object 
 4   Note     119 non-null    float64
 5   Avis     119 non-null    float64
 6   Prix     104 non-null    object 
dtypes: float64(2), object(5)
memory usage: 7.0+ KB


,Note,Avis
count,119.000000,119.000000
mean,8.331933,613.302521
std,0.762237,1141.624345
min,3.600000,1.000000
25%,8.000000,100.000000
50%,8.400000,285.000000
75%,8.800000,674.500000
max,9.800000,9616.000000


In [44]:
df.duplicated().sum()  # vérifier les doublons

np.int64(0)

In [46]:
df.dtypes

Ville       object
Nom         object
Adresse     object
Étoiles     object
Note       float64
Avis       float64
Prix        object
dtype: object

In [47]:
df.isna().sum()

Ville       0
Nom         0
Adresse     0
Étoiles    19
Note        7
Avis        7
Prix       22
dtype: int64

In [48]:
df.columns = (df.columns
              .str.lower() # mettre en minuscules
              .str.strip() # enlever les espaces en trop
              .str.replace("é", "e") # remplacer les accents
)
#Vérifier le nouveau nom des colonnes
df.columns

Index(['ville', 'nom', 'adresse', 'etoiles', 'note', 'avis', 'prix'], dtype='object')

In [50]:
#Convertir prix en float
df['prix'] = (df['prix'].astype(str).str.replace('€', '').str.replace(',', '.').str.strip())
#Remplacer les "" par NAN pour que pandas les reconaisse comme des valeurs manquantes
df["prix"] = df["prix"].replace("", pd.NA).astype(float)
#Vérifier la colomnne prix
df["prix"].head()

0    12.0
1    31.0
2    32.0
3    31.0
4    14.0
Name: prix, dtype: float64

In [51]:
import numpy as np
import pandas as pd
#Remplacer les valeurs manquantes dans la colonne "Étoiles" par 0
df["etoiles"] = df["etoiles"].fillna(0)
#Remplacer les avis manquants par 0
df["avis"] = df["avis"].fillna(0)
# remplir les prix manquants avec la médiane par ville
df[prix] = df.groupby("ville")["prix"].transform(lambda x: x.fillna(x.median()))

#Vérifier les modifications
df.isna().sum()

c:\Users\Utilisateur\Documents\Projet Recap-Problematic en Cambodge\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


ville       0
nom         0
adresse     0
etoiles     0
note        7
avis        0
prix       22
20 €        5
dtype: int64

In [52]:
df.columns.tolist()


['ville', 'nom', 'adresse', 'etoiles', 'note', 'avis', 'prix', '20 €']

In [55]:
print((df["prix"] == df["20 €"]).all())

False


In [56]:
df["prix"] = (
    df["20 €"]
    .astype(str)
    .str.replace("€", "")
    .str.replace(",", ".")
    .str.strip()
    .replace("", np.nan)
    .astype(float)
)
df = df.drop(columns=["20 €"])
df.head(10)

,ville,nom,adresse,etoiles,note,avis,prix
0,Siem Reap,Onederz Hostel Siem Reap,"Siem Reap centre,Siem Reap - Voir sur la carte",3 étoiles,9.3,3456.0,12.0
1,Siem Reap,Pandora Suite D'Angkor,"Kouk Chak,Siem Reap - Voir sur la carte",5 étoiles,9.2,1004.0,31.0
2,Siem Reap,Elysium Suite La Residence,"Sala Kamreuk,Siem Reap - Voir sur la carte",4 étoiles,9.1,439.0,32.0
3,Siem Reap,Grand Venus La Residence,"Kouk Chak,Siem Reap - Voir sur la carte",0,9.0,1040.0,31.0
4,Siem Reap,Lub d Siem Reap,"Siem Reap centre,Siem Reap - Voir sur la carte",4 étoiles,9.3,3129.0,14.0
5,Siem Reap,Bopha Wat Bo Residence,"Siem Reap centre,Siem Reap - Voir sur la carte",5 étoiles,9.2,285.0,29.0
6,Siem Reap,Indra Angkor Residence,"Sala Kamreuk,Siem Reap - Voir sur la carte",4 étoiles,8.7,597.0,25.0
7,Siem Reap,Darling Pub Hostel,"Siem Reap centre,Siem Reap - Voir sur la carte",3 étoiles,8.9,894.0,19.0
8,Siem Reap,VPlus Hotel,"Siem Reap centre,Siem Reap - Voir sur la carte",3 étoiles,9.4,1468.0,10.0
9,Siem Reap,Tropical Breeze Guesthouse,"Siem Reap centre,Siem Reap - Voir sur la carte",2 étoiles,8.2,550.0,14.0


In [57]:
df.isna().sum()

ville      0
nom        0
adresse    0
etoiles    0
note       7
avis       0
prix       5
dtype: int64

In [59]:
print("Toutes les lignes avec au moins une valeur manquante :")
df[df.isna().any(axis=1)]

Toutes les lignes avec au moins une valeur manquante :


,ville,nom,adresse,etoiles,note,avis,prix
58,Kampong Cham,Toek Chha Temple Resort,"Kampong Cham,Kampong Cham - Voir sur la carte",1 étoile,NaN,0.0,23.5
63,Kampong Thom,Vvillage,"Balang,Kampong Thom - Voir sur la carte",0,NaN,0.0,33.0
69,Koh Kong,Tatai Nature Resort,"Phum Ti Bei,Koh Kong - Voir sur la carte",4 étoiles,NaN,0.0,49.0
83,Preah Vihear,Family Motel,"Thmei,Preah Vihear - Voir sur la carte",0,NaN,0.0,25.0
89,Stung Treng,Sampheap Guesthouse,"Mondol Bei,Stung Treng - Voir sur la carte",0,NaN,0.0,13.5
90,Stung Treng,Four Rivers Hotel,"Mondol Bei,Stung Treng - Voir sur la carte",4 étoiles,NaN,0.0,13.5
91,Takeo,Eco Garden Residences,"Sangkat Chaom Chau,Takeo - Voir sur la carte",4 étoiles,7.9,133.0,NaN
92,Takeo,The Safari Resort,"Prey Rumdeng,Takeo - Voir sur la carte",4 étoiles,9.6,3.0,NaN
93,Takeo,Diamonich Suites Hotel,"Prey Rumdeng,Takeo - Voir sur la carte",3 étoiles,NaN,0.0,NaN
94,Takeo,Alice Villa Hotel,"Chres,Takeo - Voir sur la carte",3 étoiles,7.6,63.0,NaN


In [60]:
# Remplir 'note' manquantes par la moyenne de la ville
df["note"] = df.groupby("ville")["note"].transform(lambda x: x.fillna(x.mean()))

# Remplir 'prix' manquants par la médiane de la ville
df["prix"] = df.groupby("ville")["prix"].transform(lambda x: x.fillna(x.median()))

# Vérification
print(df.isna().sum())

ville      0
nom        0
adresse    0
etoiles    0
note       0
avis       0
prix       5
dtype: int64


c:\Users\Utilisateur\Documents\Projet Recap-Problematic en Cambodge\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [66]:
df.isna().sum()
df.dtypes

ville       object
nom         object
adresse     object
etoiles     object
note       float64
avis       float64
prix       float64
dtype: object

In [64]:
df.head()

,ville,nom,adresse,etoiles,note,avis,prix
0,Siem Reap,Onederz Hostel Siem Reap,"Siem Reap centre,Siem Reap - Voir sur la carte",3 étoiles,9.3,3456.0,12.0
1,Siem Reap,Pandora Suite D'Angkor,"Kouk Chak,Siem Reap - Voir sur la carte",5 étoiles,9.2,1004.0,31.0
2,Siem Reap,Elysium Suite La Residence,"Sala Kamreuk,Siem Reap - Voir sur la carte",4 étoiles,9.1,439.0,32.0
3,Siem Reap,Grand Venus La Residence,"Kouk Chak,Siem Reap - Voir sur la carte",0,9.0,1040.0,31.0
4,Siem Reap,Lub d Siem Reap,"Siem Reap centre,Siem Reap - Voir sur la carte",4 étoiles,9.3,3129.0,14.0


In [67]:
#convertir les colonnes en int
df["etoiles"] = df["etoiles"].astype(str).str.extract(r"(\d+)")
df["etoiles"] = df["etoiles"].fillna(0).astype(int)
df["avis"] = df["avis"].astype(int)
df["prix"] = df["prix"].astype(int)

In [68]:
df["adresse"] = df["adresse"].astype(str).str.replace(r"- ?[Vv]oir sur la carte", "", regex=True)
df.head()

,ville,nom,adresse,etoiles,note,avis,prix
0,Siem Reap,Onederz Hostel Siem Reap,"Siem Reap centre,Siem Reap",3,9.3,3456,12
1,Siem Reap,Pandora Suite D'Angkor,"Kouk Chak,Siem Reap",5,9.2,1004,31
2,Siem Reap,Elysium Suite La Residence,"Sala Kamreuk,Siem Reap",4,9.1,439,32
3,Siem Reap,Grand Venus La Residence,"Kouk Chak,Siem Reap",0,9.0,1040,31
4,Siem Reap,Lub d Siem Reap,"Siem Reap centre,Siem Reap",4,9.3,3129,14


In [304]:
# Garder pour df (hôtels)
df.to_csv("hotels.csv", index=False, encoding="utf-8")


In [307]:
df.dtypes

ville       object
nom         object
adresse     object
etoiles      int64
note       float64
avis         int64
prix         int64
dtype: object

## Scrapper le deuxiéme site de attractions 

In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager
import csv
import time
import random

# --- CSV ---
fichier_csv = "attractions_cambodge.csv"
with open(fichier_csv, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Nom", "Ville", "Durée", "Note", "Avis", "Prix", "Lien"])

# --- Configuration du navigateur ---
options = Options()
# options.add_argument("--headless")  # Peut être activé pour ne pas ouvrir la fenêtre
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.7258.155 Safari/537.36")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# --- URL principale du Cambodge ---
url = "https://www.getyourguide.com/cambodge-l169079/"
driver.get(url)
time.sleep(5)

# --- Fonction pour charger toutes les activités ---
def charger_toutes_les_activites(driver, max_scrolls=50):
    scroll_count = 0
    while scroll_count < max_scrolls:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(random.uniform(2,4))  # Attendre que le JS charge
        
        try:
            # Rechercher le bouton "Voir plus"
            voir_plus = driver.find_element(By.XPATH, "//span[text()='Voir plus']/..")
            if voir_plus.is_displayed():
                # Clic forcé avec JavaScript
                driver.execute_script("arguments[0].click();", voir_plus)
                print("➡️ Clic sur 'Voir plus' (JS)")
                time.sleep(random.uniform(2,4))
        except:
            # Plus de bouton
            break
        
        scroll_count += 1

# --- Effectuer le défilement et les clics ---
charger_toutes_les_activites(driver)

# --- Extraire toutes les activités ---
cartes = driver.find_elements(By.CSS_SELECTOR, "article[data-test-id='vertical-activity-card']")
print(f"🔎 {len(cartes)} activités trouvées")

for card in cartes:
    try:
        nom = card.find_element(By.CSS_SELECTOR, "h3[data-test-id='activity-card-title']").text
    except:
        nom = "N/A"
    try:
        duree = card.find_element(By.CSS_SELECTOR, "li.activity-attributes__attribute").text
    except:
        duree = "N/A"
    try:
        note = card.find_element(By.CSS_SELECTOR, "div.c-activity-rating__rating").text
    except:
        note = "N/A"
    try:
        avis = card.find_element(By.CSS_SELECTOR, "div.c-activity-rating__label").text
    except:
        avis = "N/A"
    try:
        prix = card.find_element(By.CSS_SELECTOR, "span.activity-price__text-price").text
    except:
        prix = "N/A"
    try:
        lien = card.find_element(By.CSS_SELECTOR, "a[data-test-id='vertical-activity-card-link']").get_attribute("href")
    except:
        lien = "N/A"

    # Enregistrement dans le CSV
    with open(fichier_csv, mode="a", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow([nom, "Cambodge", duree, note, avis, prix, lien])

driver.quit()
print(f"✅ Extraction terminée. Les données sont dans {fichier_csv}")


➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'Voir plus' (JS)
➡️ Clic sur 'V

In [186]:
import pandas as pd
df_attractions = pd.read_csv("attractions_cambodge.csv")
df_attractions.head(5)  

,Nom,Ville,Durée,Note,Avis,Prix,Lien
0,Siem Reap : Angkor Wat : visite en petit group...,Cambodge,8 - 9 heures • Prise en charge disponible,"4,9",(6 779),13 €,https://www.getyourguide.com/fr-fr/siem-reap-l...
1,Au départ de Siem Reap : Tour en bateau du vil...,Cambodge,6 heures • Petit groupe • Prise en charge disp...,"4,9",(4 952),17 €,https://www.getyourguide.com/fr-fr/siem-reap-l...
2,Siem Reap : Visite de 2 jours à Angkor Wat ave...,Cambodge,2 jours • Petit groupe • Prise en charge dispo...,"4,9",(3 481),25 €,https://www.getyourguide.com/fr-fr/siem-reap-l...
3,Au départ de HCM : Circuit de 2 jours dans le ...,Cambodge,2 jours • Prise en charge disponible,"4,5",(1 299),73 €,https://www.getyourguide.com/fr-fr/ho-chi-minh...
4,Phnom Penh : Les champs de la mort et le musée...,Cambodge,4 heures • Prise en charge disponible,"4,8",(2 613),16 €,https://www.getyourguide.com/fr-fr/phnom-penh-...


In [187]:
df_attractions.info()
df_attractions.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 822 entries, 0 to 821
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Nom     822 non-null    object
 1   Ville   822 non-null    object
 2   Durée   821 non-null    object
 3   Note    771 non-null    object
 4   Avis    658 non-null    object
 5   Prix    822 non-null    object
 6   Lien    822 non-null    object
dtypes: object(7)
memory usage: 45.1+ KB


,Nom,Ville,Durée,Note,Avis,Prix,Lien
count,822,822,821,771,658,822,822
unique,804,1,202,19,153,180,822
top,Visite d'Angkor Wat en tuk-tuk avec chauffeur ...,Cambodge,8 heures • Prise en charge disponible,5,(1),38 €,https://www.getyourguide.com/fr-fr/siem-reap-l...
freq,2,822,60,345,108,34,1


In [188]:
df_attractions.dtypes

Nom      object
Ville    object
Durée    object
Note     object
Avis     object
Prix     object
Lien     object
dtype: object

In [189]:
# vérifier les doublons
df_attractions.duplicated().sum()  

np.int64(0)

In [190]:
#Vérifier les valeur manquants
df_attractions.isna().sum()

Nom        0
Ville      0
Durée      1
Note      51
Avis     164
Prix       0
Lien       0
dtype: int64

In [191]:
#Vérifier les lignes avec au moins une valeur manquante dans la colonne durée
print(df_attractions[df_attractions["Durée"].isna()])

                                                   Nom     Ville Durée Note  \
796  Sihanoukville : visite d'une jounée au village...  Cambodge   NaN  4,4   

    Avis  Prix                                               Lien  
796  NaN  55 €  https://www.getyourguide.com/fr-fr/sihanoukvil...  


In [192]:
#reemplacer les valeur manquant dans la colonne "Durée" "796"
df_attractions.loc[796, "Durée"] = "4 heures"

In [193]:
#Nettoyer la colonnes avis
df_attractions["Avis"] = df_attractions["Avis"].astype(str).str.replace(r"[^\d,\.]", "", regex=True).str.replace(",", ".", regex=False)
df_attractions["Avis"] = df_attractions["Avis"].replace("", "0").astype(float)

In [194]:
#convertir note en float
df_attractions["Note"] = df_attractions["Note"].str.replace(",", ".").replace("", np.nan).astype(float)


In [195]:
#Remplir la note manquante par la moyenne des notes existentes
moyenne_note = df_attractions["Note"].mean()
df_attractions["Note"] = df_attractions["Note"].fillna(moyenne_note)

In [196]:
# Calculer la moyenne des avis (en excluant les 0)
moyenne_avis = df_attractions.loc[df_attractions["Avis"] > 0, "Avis"].mean()

# Remplacer les valeurs manquantes ou 0 par la moyenne
df_attractions["Avis"] = df_attractions["Avis"].replace(0, moyenne_avis)


In [197]:
#Convertir la colonne avis en int
df_attractions["Avis"] = df_attractions["Avis"].replace("", "0").astype(int)

In [198]:
#Supprimer les texte inutiles dans la colonne durée
# Nettoyer la colonne Durée
df_attractions["Durée"] = (
    df_attractions["Durée"]
    .astype(str)
    .str.extract(r"([\d,\.]+\s*(?:heures?|jours?|minutes?))", expand=False)
)



In [199]:
df_attractions.isna().sum()

Nom      0
Ville    0
Durée    2
Note     0
Avis     0
Prix     0
Lien     0
dtype: int64

In [200]:
print(df_attractions[df_attractions["Durée"].isna()])

                                                   Nom     Ville Durée  Note  \
253  Angkor Wat Découvrez les temples les plus embl...  Cambodge   NaN   5.0   
715    Visite privée d'Angkor Wat au coucher du soleil  Cambodge   NaN   4.8   

     Avis  Prix                                               Lien  
253     3  20 €  https://www.getyourguide.com/fr-fr/siem-reap-l...  
715   100  81 €  https://www.getyourguide.com/fr-fr/cambodge-l1...  


In [ ]:
#Remplir les valeurs manquantes dans la colonne durée
df_attractions.loc[[253,715], "Durée"] = ["8 heures","1 jour"]

In [204]:
#Vérifier les modifications
df_attractions.dtypes

Nom       object
Ville     object
Durée     object
Note     float64
Avis       int64
Prix      object
Lien      object
dtype: object

In [205]:
df_attractions["Prix"].head(10)

0    13 €
1    17 €
2    25 €
3    73 €
4    16 €
5     8 €
6    43 €
7     8 €
8    41 €
9    16 €
Name: Prix, dtype: object

In [206]:
# Nettoyer la colonne Prix
df_attractions["Prix"] = (
    df_attractions["Prix"]
    .str.replace("€", "", regex=False)      # enlever le symbole €
    .str.replace(",", ".", regex=False)     # remplacer virgule par point si existe
    .str.replace(r"\s+", "", regex=True)    # enlever les espaces inutiles
    .astype(float)                          # convertir en float
)



In [207]:
df_attractions.isna().sum()

Nom      0
Ville    0
Durée    0
Note     0
Avis     0
Prix     0
Lien     0
dtype: int64

In [208]:
df_attractions.dtypes

Nom       object
Ville     object
Durée     object
Note     float64
Avis       int64
Prix     float64
Lien      object
dtype: object

In [125]:
df_attractions.head(10)

,Nom,Ville,Durée,Note,Avis,Prix,Lien
0,Siem Reap : Angkor Wat : visite en petit group...,Cambodge,9 heures,4.9,6779,13.0,https://www.getyourguide.com/fr-fr/siem-reap-l...
1,Au départ de Siem Reap : Tour en bateau du vil...,Cambodge,6 heures,4.9,4952,17.0,https://www.getyourguide.com/fr-fr/siem-reap-l...
2,Siem Reap : Visite de 2 jours à Angkor Wat ave...,Cambodge,2 jours,4.9,3481,25.0,https://www.getyourguide.com/fr-fr/siem-reap-l...
3,Au départ de HCM : Circuit de 2 jours dans le ...,Cambodge,2 jours,4.5,1299,73.0,https://www.getyourguide.com/fr-fr/ho-chi-minh...
4,Phnom Penh : Les champs de la mort et le musée...,Cambodge,4 heures,4.8,2613,16.0,https://www.getyourguide.com/fr-fr/phnom-penh-...
5,"Battambang : Tuk-Tuk, train de bambou, grottes...",Cambodge,12 heures,4.9,1030,8.0,https://www.getyourguide.com/fr-fr/battambang-...
6,Siem Reap : 1 jour de visite d'Angkor avec gui...,Cambodge,8 heures,4.9,147,43.0,https://www.getyourguide.com/fr-fr/siem-reap-l...
7,Phnom Penh : Croisière au coucher du soleil av...,Cambodge,"1,5 heures",4.5,817,8.0,https://www.getyourguide.com/fr-fr/phnom-penh-...
8,Montagne Kulen : Visite en petit groupe et déj...,Cambodge,8 heures,4.9,1177,41.0,https://www.getyourguide.com/fr-fr/siem-reap-l...
9,Siem Reap : Kampong Phluk Floating Village Tou...,Cambodge,6 heures,4.9,3013,16.0,https://www.getyourguide.com/fr-fr/kampong-phl...


In [209]:
#Supprimer la colonne lien
df_attractions = df_attractions.drop(columns=["Lien"])


In [210]:
# Extraire la partie numérique + unité
df_attractions[["nombre", "unite"]] = df_attractions["Durée"].str.extract(r"([\d,\.]+)\s*(minutes?|heures?|jours?)")

# Remplacer les virgules par des points pour pouvoir convertir en float
df_attractions["nombre"] = df_attractions["nombre"].str.replace(",", ".", regex=False).astype(float)

# Conversion en heures
df_attractions["Durée_heures"] = np.where(
    df_attractions["unite"].str.contains("minute", case=False, na=False),
    df_attractions["nombre"] / 60,  # convertir minutes → heures
    np.where(
        df_attractions["unite"].str.contains("jour", case=False, na=False),
        df_attractions["nombre"] * 24,  # convertir jours → heures
        df_attractions["nombre"]  # déjà en heures
    )
)

# Optionnel : supprimer les colonnes temporaires
df_attractions = df_attractions.drop(columns=["nombre", "unite"])


In [211]:
#Renommer la colonne durée_heures en durée et supprimer l'ancienne colonne durée
df_attractions["Durée"] = df_attractions["Durée_heures"]
df_attractions = df_attractions.drop(columns=["Durée_heures"])


In [212]:
df_attractions["Nom"].head(25)

0     Siem Reap : Angkor Wat : visite en petit group...
1     Au départ de Siem Reap : Tour en bateau du vil...
2     Siem Reap : Visite de 2 jours à Angkor Wat ave...
3     Au départ de HCM : Circuit de 2 jours dans le ...
4     Phnom Penh : Les champs de la mort et le musée...
5     Battambang : Tuk-Tuk, train de bambou, grottes...
6     Siem Reap : 1 jour de visite d'Angkor avec gui...
7     Phnom Penh : Croisière au coucher du soleil av...
8     Montagne Kulen : Visite en petit groupe et déj...
9     Siem Reap : Kampong Phluk Floating Village Tou...
10    Siem Reap : excursion à la montagne Kulen, à B...
11    Ho Chi Minh Ville : 2 jours de vélo et de kaya...
12    Kampot : Visite d'une ferme de poivre, d'un ch...
13    Siem Reap : Visite d'une demi-journée du villa...
14    Siem Reap : Spectacle d'Apsara avec dîner et p...
15    Depuis Siem Reap : Expérience au Sanctuaire de...
16    Siem Reap : Visite privée en tuk-tuk d'Angkor Wat
17    Siem Reap : Phare, le spectacle de cirque 

In [214]:
df_attractions.columns = (df_attractions.columns
              .str.lower() # mettre en minuscules
              .str.strip() # enlever les espaces en trop
              .str.replace("é", "e") # remplacer les accents
)
#Vérifier le nouveau nom des colonnes
df_attractions.columns

Index(['nom', 'ville', 'duree', 'note', 'avis', 'prix'], dtype='object')

In [215]:
#Créer une nouvelle copie du dataframe
df_test = df_attractions.copy()

In [216]:
#Fonction d'extraction prudente (seulement cas sûrs)
import re

def extraire_ville_debut(nom):
    if not isinstance(nom, str):
        return None
    nom = nom.strip()
    # Cas "Au départ de X : ..." ou "Depuis X : ..."
    m = re.match(r'^(?:Au départ de |Au depart de |Depuis |Depuis la |Depuis le |Depuis l\')(.+?)(?:\s*:)', nom, flags=re.I)
    if m:
        return m.group(1).strip()
    # Cas "Ville : ..." (texte avant le premier ":")
    m2 = re.match(r'^([^:]{1,60}?)\s*:', nom)  # limite 60 chars pour éviter capturer de longues phrases
    if m2:
        return m2.group(1).strip()
    return None

In [217]:
#Appliquer
df_test['ville_extr1'] = df_test['nom'].apply(extraire_ville_debut)

In [219]:
print("Nombre de lignes avec extraction au début :", df_test['ville_extr1'].notna().sum())
print("\nTop villes extraites (exemples) :")
print(df_test['ville_extr1'].value_counts().head(30))

Nombre de lignes avec extraction au début : 424

Top villes extraites (exemples) :
ville_extr1
Siem Reap                                                 219
Phnom Penh                                                 68
Angkor Wat                                                 17
Kampot                                                     10
HCM                                                        10
Ho Chi Minh                                                 9
Battambang                                                  9
Hô Chi Minh-Ville                                           3
Ho Chi Minh Ville                                           3
Sihanoukville                                               3
De Siem Reap                                                3
Journée complète                                            3
Montagne Kulen                                              2
Krong Siem Reap                                             2
Retour aux sources                   

In [220]:
display(df_test.loc[df_test['ville_extr1'].notna(), ['nom', 'ville_extr1']].head(30))

,nom,ville_extr1
0,Siem Reap : Angkor Wat : visite en petit group...,Siem Reap
1,Au départ de Siem Reap : Tour en bateau du vil...,Siem Reap
2,Siem Reap : Visite de 2 jours à Angkor Wat ave...,Siem Reap
3,Au départ de HCM : Circuit de 2 jours dans le ...,HCM
4,Phnom Penh : Les champs de la mort et le musée...,Phnom Penh
5,"Battambang : Tuk-Tuk, train de bambou, grottes...",Battambang
6,Siem Reap : 1 jour de visite d'Angkor avec gui...,Siem Reap
7,Phnom Penh : Croisière au coucher du soleil av...,Phnom Penh
8,Montagne Kulen : Visite en petit groupe et déj...,Montagne Kulen
9,Siem Reap : Kampong Phluk Floating Village Tou...,Siem Reap


In [221]:
# Isoler les lignes sans ville détectée
df_sans_ville = df_test[df_test['ville_extr1'].isna()].copy()

print(f"Lignes sans ville détectée : {len(df_sans_ville)}")
print(df_sans_ville[['nom']].head(20))  # afficher un échantillon


Lignes sans ville détectée : 398
                                                  nom
20  Phnom Penh à l'île de la soie en tuk-tuk avec ...
24  Transfert routier Phnom Penh - Siem Reap plus ...
26          Circuit en quad à la campagne à Siem Reap
28  Excursion dans la forêt d'éléphants de Kulen a...
29  Visite privée d'une journée des temples d'Angk...
33  Angkor Wat 2 jours complexe Lever et coucher d...
34  Mont Oudong - Ancienne capitale de Phnom Penh ...
40  Siem Reap Floating Village Kampong Phluk Sunse...
41  Visite de la ferme de soie Lotus à Siem Reap a...
43  Spectacle d'Apsara avec dîner, prise en charge...
44  Banteay Srei, Beng Mealea et la montagne Kulen...
45                 Angkor, une montgolfière étonnante
46  Cascade de Phnom Kulen et les 1 000 lingas sac...
48  Exploration de Banteay Srei, de la cascade de ...
52           Visite de 9 lieux en 3 heures en tuk-tuk
59  Représentation théâtrale d'Apsara, avec dîner ...
66  Location de voiture privée à Siem Reap avec c

In [222]:
# Dictionnaire des villes à chercher dans 'nom'
villes_a_chercher = [
    "Siem Reap", "Phnom Penh", "Kampot", "Battambang", 
    "Kratie", "Sihanoukville", "Preah Vihear", "Koh Ker", 
    "Beng Mealea", "Montagne Kulen", "Phnom Kulen", "Oudong"
]

# Compléter les villes manquantes en fonction des mots-clés trouvés dans 'nom'
def detecter_ville(nom, ville_existante):
    if pd.notna(ville_existante):
        return ville_existante  # ne pas écraser ce qui existe déjà
    for ville in villes_a_chercher:
        if ville.lower() in nom.lower():
            return ville
    return None

df_test['ville_extr1'] = df_test.apply(
    lambda row: detecter_ville(row['nom'], row['ville_extr1']),
    axis=1
)

print("Après détection complémentaire :", df_test['ville_extr1'].notna().sum(), "lignes avec ville")
print(df_test['ville_extr1'].value_counts().head(20))


Après détection complémentaire : 606 lignes avec ville
ville_extr1
Siem Reap             295
Phnom Penh            119
Kampot                 23
Battambang             17
Angkor Wat             17
HCM                    10
Beng Mealea            10
Koh Ker                10
Ho Chi Minh             9
Montagne Kulen          7
Sihanoukville           7
Preah Vihear            4
De Siem Reap            3
Ho Chi Minh Ville       3
Journée complète        3
Hô Chi Minh-Ville       3
Retour aux sources      2
Demi-journée            2
Kratie                  2
Krong Siem Reap         2
Name: count, dtype: int64


In [225]:
# Liste des mots à remplacer si ce ne sont pas de vraies villes
faux_positifs = ["Journée complète", "Retour aux sources", "Demi-journée"]

# Liste élargie de villes connues pour les rechercher dans le texte
villes_connues = [
    "Siem Reap", "Phnom Penh", "Kampot", "Battambang", "Kratie", 
    "Sihanoukville", "Preah Vihear", "Krong Siem Reap", "Kep",
    "Ho Chi Minh", "Ho Chi Minh Ville", "HCM", "Hô Chi Minh-Ville",
    "Montagne Kulen", "Parc de Phnom Kulen", "Beng Mealea", "Koh Ker"
]

def detect_ville_dans_texte(texte):
    """Cherche une ville connue dans le texte"""
    for ville in villes_connues:
        if ville.lower() in texte.lower():
            return ville
    return None

# Correction ciblée des faux positifs
for idx, row in df_test.iterrows():
    if row['ville_extr1'] in faux_positifs:
        ville_detectee = detect_ville_dans_texte(row['nom'])
        if ville_detectee:
            df_test.at[idx, 'ville_extr1'] = ville_detectee
        else:
            df_test.at[idx, 'ville_extr1'] = None

# Vérification
print("Nombre de lignes avec ville après correction :", df_test['ville_extr1'].notna().sum())
print(df_test[df_test['ville_extr1'].isna()].head(10))


Nombre de lignes avec ville après correction : 601
                                                   nom     ville  duree  note  \
28   Excursion dans la forêt d'éléphants de Kulen a...  Cambodge    6.0   4.9   
33   Angkor Wat 2 jours complexe Lever et coucher d...  Cambodge   48.0   4.9   
43   Spectacle d'Apsara avec dîner, prise en charge...  Cambodge    2.5   4.6   
45                  Angkor, une montgolfière étonnante  Cambodge    0.5   4.4   
52            Visite de 9 lieux en 3 heures en tuk-tuk  Cambodge    3.0   4.9   
59   Représentation théâtrale d'Apsara, avec dîner ...  Cambodge    2.5   4.8   
87   La campagne comprend une ferme poivrière, une ...  Cambodge    5.0   4.9   
100  Excursion en tuk-tuk à Silk Island et à la mon...  Cambodge    8.0   5.0   
109  Excursion d'une journée complète avec excursio...  Cambodge    7.0   4.4   
112  Une journée TukTuk Handicrafts Bambootrain Kil...  Cambodge   12.0   4.9   

     avis   prix ville_extr1  
28    121  110.0        No

In [226]:
# Dictionnaire mots-clés → ville
mots_cles_villes = {
    "Angkor": "Siem Reap",
    "Kulen": "Siem Reap",
    "TukTuk": "Siem Reap",
    "Floating Village": "Siem Reap",
    "Phnom Penh": "Phnom Penh",
    "Kampot": "Kampot",
    "Battambang": "Battambang",
    "Preah Vihear": "Preah Vihear",
    "Sihanoukville": "Sihanoukville"
}

# Fonction pour assigner une ville selon les mots-clés
def recuperer_ville_par_keywords(nom):
    if pd.isna(nom):
        return None
    for mot, ville in mots_cles_villes.items():
        if mot.lower() in nom.lower():
            return ville
    return None

# Appliquer uniquement sur les lignes où ville_extr1 est None
df_test['ville_extr1'] = df_test.apply(
    lambda row: recuperer_ville_par_keywords(row['nom']) if pd.isna(row['ville_extr1']) else row['ville_extr1'],
    axis=1
)

# Vérifier combien de lignes sont encore None
print("Lignes sans ville après détection par mots-clés :", df_test['ville_extr1'].isna().sum())


Lignes sans ville après détection par mots-clés : 111


In [234]:
# Afficher uniquement la colonne 'nom' pour les lignes sans ville détectée
print(df_test[df_test['ville_extr1'].isna()]['nom'].head(20))  # 20 premières lignes


43     Spectacle d'Apsara avec dîner, prise en charge...
52              Visite de 9 lieux en 3 heures en tuk-tuk
59     Représentation théâtrale d'Apsara, avec dîner ...
87     La campagne comprend une ferme poivrière, une ...
100    Excursion en tuk-tuk à Silk Island et à la mon...
109    Excursion d'une journée complète avec excursio...
116    Bantey Srei, Kbal Spean et musée des mines ter...
129    Excursion en kayak autour de la cathédrale ver...
134     Excursion en kayak autour de la cathédrale verte
137    Une journée à Taprohm Bati - Chisor Temple - P...
141                        Cours de cuisine cambodgienne
146    Découvrez une cascade cachée, faites un trek d...
151    Dîner spectacle de danse Apsara, prise en char...
159    Explorer le charmant village flottant de Kampo...
163    Excursion d'une journée au Phnom Tamao Wildlif...
166    Demi-journée : pont suspendu, train de bambou,...
169    Visite de 2 jours du marché flottant de Cai Ra...
170    Parc national de Bokor e

In [235]:
# Dictionnaire mots-clés → ville
mots_cles_villes = {
    "Siem Reap": ["Siem Reap", "Angkor", "Banteay Srei", "Kbal Spean", "Taprohm", "Bati", "Chisor"],
    "Phnom Penh": ["Phnom Penh", "Phnom Tamao", "Silk Island"],
    "Kampot": ["Bokor", "Kampot"],
    "Battambang": ["Bambootrain", "Handicrafts Bambootrain"],
    "Koh Ker": ["Koh Ker"],
    "Preah Vihear": ["Preah Vihear"],
}

# Fonction pour assigner la ville en fonction des mots-clés
def detect_ville_par_mots_cles(nom):
    if pd.isna(nom):
        return None
    for ville, mots in mots_cles_villes.items():
        for mot in mots:
            if mot.lower() in nom.lower():
                return ville
    return None

# Appliquer uniquement sur les lignes encore sans ville
df_test['ville_extr2'] = df_test.apply(
    lambda row: row['ville_extr1'] if pd.notna(row['ville_extr1']) else detect_ville_par_mots_cles(row['nom']),
    axis=1
)

# Vérifier combien de lignes ont maintenant une ville
print("Nombre de lignes avec ville après détection complémentaire :", df_test['ville_extr2'].notna().sum())

# Afficher les lignes encore sans ville
print(df_test[df_test['ville_extr2'].isna()]['nom'])


Nombre de lignes avec ville après détection complémentaire : 723
43     Spectacle d'Apsara avec dîner, prise en charge...
52              Visite de 9 lieux en 3 heures en tuk-tuk
59     Représentation théâtrale d'Apsara, avec dîner ...
87     La campagne comprend une ferme poivrière, une ...
109    Excursion d'une journée complète avec excursio...
                             ...                        
777    Village flottant de Kampong Phluk et excursion...
780        Circuit de deux jours dans le delta du Mékong
792    Excursion à Sambor Preikuk, Kuhak Nokor et Spi...
800           Location de fauteuils roulants au Cambodge
820    Visite de dégustation de plats locaux et de co...
Name: nom, Length: 99, dtype: object


In [242]:
# Liste de villes et lieux connus au Cambodge et Vietnam
villes_connues = [
    "Siem Reap", "Phnom Penh", "Kampot", "Battambang","Kratie",
    "Kep", "Sihanoukville", "Koh Ker", "Beng Mealea","Preah Vihear",
    "Tonle Sap","Koh Ker","Hô Chi Minh Ville", "Montagne Kulen", "Banteay Srei",
    "Preah Vihear", "Silk Island", "Ho Chi Minh", "HCM",
    "Ho Chi Minh Ville", "Hô Chi Minh-Ville",
    "Phnom Kulen", "Angkor Wat", "Kampong Phluk",
    "Bokor", "Parc de Phonom Kulen", "Beng Mealea", "Île de la Soie"
]


In [243]:
def detecter_ville(nom):
    if pd.isna(nom):
        return None
    nom_lower = nom.lower()
    for ville in villes_connues:
        if ville.lower() in nom_lower:
            return ville
    return None  # pour l'instant, si rien trouvé


In [244]:
# Créer une nouvelle colonne pour les villes détectées
df_test['ville_detectee'] = df_test.apply(
    lambda row: detecter_ville(row['nom']) if pd.isna(row['ville_extr1']) else row['ville_extr1'],
    axis=1
)

# Vérifier combien de lignes restent encore sans ville
print("Nombre de lignes sans ville après détection complémentaire :", df_test['ville_detectee'].isna().sum())


Nombre de lignes sans ville après détection complémentaire : 89


In [245]:
# Afficher seulement les lignes sans ville détectée
lignes_sans_ville = df_test[df_test['ville_detectee'].isna()]['nom']
print(lignes_sans_ville)


43     Spectacle d'Apsara avec dîner, prise en charge...
52              Visite de 9 lieux en 3 heures en tuk-tuk
59     Représentation théâtrale d'Apsara, avec dîner ...
87     La campagne comprend une ferme poivrière, une ...
109    Excursion d'une journée complète avec excursio...
                             ...                        
738    Excursion en jeep au coucher du soleil dans la...
780        Circuit de deux jours dans le delta du Mékong
792    Excursion à Sambor Preikuk, Kuhak Nokor et Spi...
800           Location de fauteuils roulants au Cambodge
820    Visite de dégustation de plats locaux et de co...
Name: nom, Length: 89, dtype: object


In [246]:
# Liste de villes principales au Cambodge
villes_connues = ["Siem Reap", "Phnom Penh", "Kampot", "Battambang", 
                  "Sihanoukville", "Preah Vihear", "Koh Ker", "Montagne Kulen",
                  "Kep", "Kampong Phluk", "Silk Island", "Phnom Tamao"]

# Détection simple : chercher si un des mots-clés est dans le nom
for ville in villes_connues:
    df_test.loc[df_test['ville_detectee'].isna() & df_test['nom'].str.contains(ville, case=False, na=False), 'ville_detectee'] = ville

# Vérifier combien restent encore non détectées
print("Lignes sans ville après cette détection :", df_test['ville_detectee'].isna().sum())


Lignes sans ville après cette détection : 88


In [248]:
# Pour les lignes restantes sans ville détectée, mettre Cambodge
df_test['ville_detectee'] = df_test['ville_detectee'].fillna("Cambodge")


In [251]:
df_test['ville_finale_corrigee'] = (
    df_test['ville_detectee']
    .combine_first(df_test['ville_extr2'])
    .combine_first(df_test['ville_extr1'])
    .fillna("Cambodge")
)

# Vérifier combien de lignes ont encore Cambodge
print("Lignes avec ville par défaut Cambodge :", 
      (df_test['ville_finale_corrigee'] == "Cambodge").sum())

# Afficher un échantillon
print(df_test[['nom', 'ville_finale_corrigee']].head(20))


Lignes avec ville par défaut Cambodge : 89
                                                  nom ville_finale_corrigee
0   Siem Reap : Angkor Wat : visite en petit group...             Siem Reap
1   Au départ de Siem Reap : Tour en bateau du vil...             Siem Reap
2   Siem Reap : Visite de 2 jours à Angkor Wat ave...             Siem Reap
3   Au départ de HCM : Circuit de 2 jours dans le ...                   HCM
4   Phnom Penh : Les champs de la mort et le musée...            Phnom Penh
5   Battambang : Tuk-Tuk, train de bambou, grottes...            Battambang
6   Siem Reap : 1 jour de visite d'Angkor avec gui...             Siem Reap
7   Phnom Penh : Croisière au coucher du soleil av...            Phnom Penh
8   Montagne Kulen : Visite en petit groupe et déj...        Montagne Kulen
9   Siem Reap : Kampong Phluk Floating Village Tou...             Siem Reap
10  Siem Reap : excursion à la montagne Kulen, à B...             Siem Reap
11  Ho Chi Minh Ville : 2 jours de vélo et de

In [253]:
lignes_cambodge = df_test[df_test['ville_finale_corrigee'] == "Cambodge"]
print(lignes_cambodge['nom'].head(20))  # afficher un échantillon


43     Spectacle d'Apsara avec dîner, prise en charge...
52              Visite de 9 lieux en 3 heures en tuk-tuk
59     Représentation théâtrale d'Apsara, avec dîner ...
87     La campagne comprend une ferme poivrière, une ...
109    Excursion d'une journée complète avec excursio...
116    Bantey Srei, Kbal Spean et musée des mines ter...
129    Excursion en kayak autour de la cathédrale ver...
134     Excursion en kayak autour de la cathédrale verte
137    Une journée à Taprohm Bati - Chisor Temple - P...
141                        Cours de cuisine cambodgienne
146    Découvrez une cascade cachée, faites un trek d...
151    Dîner spectacle de danse Apsara, prise en char...
166    Demi-journée : pont suspendu, train de bambou,...
169    Visite de 2 jours du marché flottant de Cai Ra...
171    Visite privée du sanctuaire des éléphants et d...
198    Sanctuaire des éléphants au Cambodge, prise en...
213    Spectacle d'Apsara avec dîner buffet et prise ...
221    Palais royal, musée nati

In [255]:
corrections = {
    43: "Siem Reap",
    52: "Siem Reap",
    59: "Siem Reap",
    87: "Kampot",
    109: "Phnom Penh",
    116: "Siem Reap",
    129: "Siem Reap",
    134: "Siem Reap",
    137: "Siem Reap",
    141: "Phnom Penh",
    146: "Siem Reap",
    151: "Siem Reap",
    166: "Battambang",
    169: "Siem Reap",
    171: "Phnom Penh",
    198: "Phnom Penh",
    213: "Siem Reap",
    221: "Phnom Penh",
    236: "Siem Reap",
    254: "Siem Reap"
}

df_test['ville_finale_corrigee'] = df_test.apply(
    lambda row: corrections[row.name] if row.name in corrections else row['ville_finale_corrigee'],
    axis=1
)

# Vérification
lignes_cambodge = df_test[df_test['ville_finale_corrigee'] == "Cambodge"]
print(lignes_cambodge['nom'].head(20))  # afficher un échantillon

283    Exploration du village et de la campagne à vél...
286    Coucher de soleil sur le village flottant de K...
288    Excursion en tuk tuk local à la campagne et à ...
297    Représentation théâtrale d'Apsara, avec dîner ...
300     3 heures de visite en tuk-tuk dans les coulisses
317    Visite en petit groupe à vélo dans la campagne...
325    Visite artisanale/papier de riz/vin de riz/gro...
329    Excursion au Vietnam de 15 jours et 14 nuits e...
331    Visite du sanctuaire des éléphants du Cambodge...
340    Train de bambous. Grotte de Phnom Sampove. Gro...
352                   03 heures de visites à la campagne
357    Grand cercle avec coucher de soleil Tuk-Tuk privé
358                    Visite du site historique d'Udong
367    S21 et visite des champs de la mort avec déjeu...
385    Visite du village flottant avec repas khmer et...
404    Le cirque cambodgien et le marché nocturne de ...
414    Dîner buffet et spectacle d'Apsara avec transf...
424    Paysages étonnants - Tra

In [256]:
# Dictionnaire des corrections manuelles : index -> ville probable
corrections_manuel = {
    283: "Siem Reap",
    286: "Kampong Phluk",
    288: "Siem Reap",
    297: "Siem Reap",
    300: "Siem Reap",
    317: "Siem Reap",
    325: "Siem Reap",
    329: "Vietnam",
    331: "Phnom Penh",
    340: "Battambang",
    352: "Siem Reap",
    357: "Siem Reap",
    358: "Phnom Penh",
    367: "Phnom Penh",
    385: "Siem Reap",
    404: "Phnom Penh",
    414: "Siem Reap",
    424: "Battambang",
    428: "Phnom Penh",
    435: "Phnom Penh"
}

# Créer la colonne ville_finale_corrigee si elle n'existe pas encore
if 'ville_finale_corrigee' not in df_test.columns:
    df_test['ville_finale_corrigee'] = df_test['ville_detectee']

# Appliquer les corrections manuelles
for idx, ville in corrections_manuel.items():
    if idx in df_test.index:
        df_test.at[idx, 'ville_finale_corrigee'] = ville

# Vérifier les lignes mises à jour
print(df_test.loc[corrections_manuel.keys(), ['nom', 'ville_finale_corrigee']])


                                                   nom ville_finale_corrigee
283  Exploration du village et de la campagne à vél...             Siem Reap
286  Coucher de soleil sur le village flottant de K...         Kampong Phluk
288  Excursion en tuk tuk local à la campagne et à ...             Siem Reap
297  Représentation théâtrale d'Apsara, avec dîner ...             Siem Reap
300   3 heures de visite en tuk-tuk dans les coulisses             Siem Reap
317  Visite en petit groupe à vélo dans la campagne...             Siem Reap
325  Visite artisanale/papier de riz/vin de riz/gro...             Siem Reap
329  Excursion au Vietnam de 15 jours et 14 nuits e...               Vietnam
331  Visite du sanctuaire des éléphants du Cambodge...            Phnom Penh
340  Train de bambous. Grotte de Phnom Sampove. Gro...            Battambang
352                 03 heures de visites à la campagne             Siem Reap
357  Grand cercle avec coucher de soleil Tuk-Tuk privé             Siem Reap

In [257]:
# Vérification
lignes_cambodge = df_test[df_test['ville_finale_corrigee'] == "Cambodge"]
print(lignes_cambodge['nom'].head(20))  # afficher un échantillon

436                 École de danse Princesse Buppha Devi
444    Cambodge : spectacle de danse traditionnelle a...
445    Les visites de la campagne comprennent des pla...
447    Visite privée de la réserve ornithologique de ...
452                  L'expérience de Jason à la campagne
454    Beng Melea, Groupe Rolous & Village flottant d...
458    Visite culinaire et cirque Phare avec transfer...
459    Visite à pied de la rivière avec collation, bo...
496    Tour en bateau et en jeep du village flottant ...
497    Delta du Mékong - Marché flottant de Cai Rang,...
518    Excursion d'une journée à Phnom Da, Ta Prohm B...
524    Journée complète : artisanat, train de bambous...
528    Visite de Phnom Bakheng au coucher du soleil, ...
531    Visite privée à Anlong Veng (forteresse des Kh...
537          1 jour Delta du Mékong - Cai Be - Vinh Long
538    Visite guidée d'une demi-journée en kayak dans...
543               Mélange de temples et de chutes d'eau.
547    Bantreay Srei, Kbal Spea

In [258]:
corrections_manuel_suite = {
    436: "Siem Reap",
    444: "Siem Reap",
    445: "Siem Reap",
    447: "Preah Vihear",
    452: "Siem Reap",
    454: "Beng Mealea",
    458: "Phnom Penh",
    459: "Siem Reap",
    496: "Kampong Phluk",
    497: "Cai Rang",
    518: "Phnom Da",
    524: "Siem Reap",
    528: "Phnom Bakheng",
    531: "Anlong Veng",
    537: "Delta du Mékong",
    538: "Kayak Tour",
    543: "Siem Reap",
    547: "Banteay Srei",
    561: "Siem Reap",
    565: "Musée de la Préhistoire"
}

# Appliquer les corrections manuelles
for idx, ville in corrections_manuel_suite.items():
    if idx in df_test.index:
        df_test.at[idx, 'ville_finale_corrigee'] = ville

# Vérifier
print(df_test.loc[corrections_manuel_suite.keys(), ['nom', 'ville_finale_corrigee']])


                                                   nom  \
436               École de danse Princesse Buppha Devi   
444  Cambodge : spectacle de danse traditionnelle a...   
445  Les visites de la campagne comprennent des pla...   
447  Visite privée de la réserve ornithologique de ...   
452                L'expérience de Jason à la campagne   
454  Beng Melea, Groupe Rolous & Village flottant d...   
458  Visite culinaire et cirque Phare avec transfer...   
459  Visite à pied de la rivière avec collation, bo...   
496  Tour en bateau et en jeep du village flottant ...   
497  Delta du Mékong - Marché flottant de Cai Rang,...   
518  Excursion d'une journée à Phnom Da, Ta Prohm B...   
524  Journée complète : artisanat, train de bambous...   
528  Visite de Phnom Bakheng au coucher du soleil, ...   
531  Visite privée à Anlong Veng (forteresse des Kh...   
537        1 jour Delta du Mékong - Cai Be - Vinh Long   
538  Visite guidée d'une demi-journée en kayak dans...   
543           

In [259]:
# Vérification
lignes_cambodge = df_test[df_test['ville_finale_corrigee'] == "Cambodge"]
print(lignes_cambodge['nom'].head(20))  # afficher un échantillon

566    Excursion privée d'une demi-journée au musée d...
571    Visite des villages flottants et coucher de so...
573         Vie nocturne trois heures visites culinaires
574    Journée complète : train de bambou original, W...
582    Visite naturelle de la ferme aux poivres. Grot...
587    Visite en petit groupe ou privée | Tunnels de ...
608    Demi-journée au Killing Field et au musée du g...
612    Croisière commentée à Kompong Phluk (village f...
613    Visite d'une demi-journée du Palais royal, de ...
620    Dîner au coucher du soleil à SiemReap en charr...
633    Le spectacle au théâtre Apsara comprend le dîn...
649    Excursion au Grand Cercle incluant Banteay Sre...
651    Découvrez l'enchantement du village flottant d...
656    Train de bambous, temple de Banann, chauve-sou...
664    Visites privées des temples sur 2 jours, déjeu...
683    Spectacle du théâtre Apsara, avec dîner et pri...
701                                       Trusty Tuk Tuk
707    Observation des oiseaux 

In [260]:
corrections_manuel_suite2 = {
    566: "Musée",
    571: "Battambang",
    573: "Siem Reap",
    574: "Siem Reap",
    582: "Kampot",
    587: "Kompong Thom",
    608: "Phnom Penh",
    612: "Kompong Phluk",
    613: "Phnom Penh",
    620: "Siem Reap",
    633: "Siem Reap",
    649: "Banteay Srei",
    651: "Battambang",
    656: "Banann",
    664: "Siem Reap",
    683: "Siem Reap",
    701: "Siem Reap",
    707: "Kampong Cham",
    710: "Siem Reap",
    712: "Village de pêcheurs"
}

# Appliquer les corrections
for idx, ville in corrections_manuel_suite2.items():
    if idx in df_test.index:
        df_test.at[idx, 'ville_finale_corrigee'] = ville

# Vérifier quelques lignes
print(df_test.loc[corrections_manuel_suite2.keys(), ['nom', 'ville_finale_corrigee']])


                                                   nom ville_finale_corrigee
566  Excursion privée d'une demi-journée au musée d...                 Musée
571  Visite des villages flottants et coucher de so...            Battambang
573       Vie nocturne trois heures visites culinaires             Siem Reap
574  Journée complète : train de bambou original, W...             Siem Reap
582  Visite naturelle de la ferme aux poivres. Grot...                Kampot
587  Visite en petit groupe ou privée | Tunnels de ...          Kompong Thom
608  Demi-journée au Killing Field et au musée du g...            Phnom Penh
612  Croisière commentée à Kompong Phluk (village f...         Kompong Phluk
613  Visite d'une demi-journée du Palais royal, de ...            Phnom Penh
620  Dîner au coucher du soleil à SiemReap en charr...             Siem Reap
633  Le spectacle au théâtre Apsara comprend le dîn...             Siem Reap
649  Excursion au Grand Cercle incluant Banteay Sre...          Banteay Srei

In [261]:
# Vérification
lignes_cambodge = df_test[df_test['ville_finale_corrigee'] == "Cambodge"]
print(lignes_cambodge['nom'].head(20))  # afficher un échantillon

714    Excursion Privée et Visite Guidée des Temps Fo...
717    Sanctuaire d'oiseaux de Prek Toal et visite du...
726    Palais royal, génocide S-21, musée national et...
735    Demi-journée privée au Killing Field et au mus...
738    Excursion en jeep au coucher du soleil dans la...
780        Circuit de deux jours dans le delta du Mékong
792    Excursion à Sambor Preikuk, Kuhak Nokor et Spi...
800           Location de fauteuils roulants au Cambodge
820    Visite de dégustation de plats locaux et de co...
Name: nom, dtype: object


In [262]:
corrections_manuel_suite3 = {
    714: "Siem Reap",
    717: "Prek Toal",
    726: "Phnom Penh",
    735: "Phnom Penh",
    738: "Siem Reap",
    780: "Delta du Mékong",
    792: "Sambor Preikuk",
    800: "Cambodge",
    820: "Siem Reap"
}

# Appliquer les corrections
for idx, ville in corrections_manuel_suite3.items():
    if idx in df_test.index:
        df_test.at[idx, 'ville_finale_corrigee'] = ville

# Vérifier quelques lignes
print(df_test.loc[corrections_manuel_suite3.keys(), ['nom', 'ville_finale_corrigee']])


                                                   nom ville_finale_corrigee
714  Excursion Privée et Visite Guidée des Temps Fo...             Siem Reap
717  Sanctuaire d'oiseaux de Prek Toal et visite du...             Prek Toal
726  Palais royal, génocide S-21, musée national et...            Phnom Penh
735  Demi-journée privée au Killing Field et au mus...            Phnom Penh
738  Excursion en jeep au coucher du soleil dans la...             Siem Reap
780      Circuit de deux jours dans le delta du Mékong       Delta du Mékong
792  Excursion à Sambor Preikuk, Kuhak Nokor et Spi...        Sambor Preikuk
800         Location de fauteuils roulants au Cambodge              Cambodge
820  Visite de dégustation de plats locaux et de co...             Siem Reap


In [263]:
df_test['nom'] = df_test['nom'].str.strip()


In [264]:
df_test['nom'] = df_test['nom'].str.replace(r'\s+', ' ', regex=True)


In [266]:
df_test.duplicated().sum()  # vérifier les doublons

np.int64(0)

In [265]:
doublons = df_test[df_test.duplicated(subset=['nom'], keep=False)]
print(doublons[['nom', 'ville_finale_corrigee']])


                                                   nom ville_finale_corrigee
3    Au départ de HCM : Circuit de 2 jours dans le ...                   HCM
45                  Angkor, une montgolfière étonnante             Siem Reap
59   Représentation théâtrale d'Apsara, avec dîner ...             Siem Reap
69            Siem Reap : Billet d'entrée à Angkor Wat             Siem Reap
127           Siem Reap : Billet d'entrée à Angkor Wat             Siem Reap
193                 Angkor, une montgolfière étonnante             Siem Reap
203  Siem Reap : bénédiction de l'eau par les boudd...             Siem Reap
210  Siem Reap : Chute d'eau de Kulen en visite privée             Siem Reap
216          Cours de cuisine cambodgienne à Siem Reap             Siem Reap
231  Visite d'Angkor Wat en tuk-tuk avec chauffeur ...             Siem Reap
252                   Sky Venture Microlight Siem Reap             Siem Reap
270  Au départ de HCM : 2 jours d'exploration du de...                   HCM

In [267]:
df_test = df_test.drop_duplicates(subset=['nom', 'ville_finale_corrigee'])


In [268]:
doublons = df_test[df_test.duplicated(subset=['nom'], keep=False)]
print(doublons[['nom', 'ville_finale_corrigee']])

Empty DataFrame
Columns: [nom, ville_finale_corrigee]
Index: []


In [272]:
lignes_cambodge = df_test[df_test['ville_finale_corrigee'] == "Cambodge"]
print(lignes_cambodge['nom'].head(20))  # afficher un échantillon

800    Location de fauteuils roulants au Cambodge
Name: nom, dtype: object


In [273]:
# Vérifier combien restent encore non détectées
print("Lignes sans ville après cette détection :", df_test['ville_finale_corrigee'].isna().sum())

Lignes sans ville après cette détection : 0


In [274]:
df_test['ville_finale_corrigee'].value_counts()

ville_finale_corrigee
Siem Reap          435
Phnom Penh         134
Kampot              25
Battambang          22
Angkor Wat          17
                  ... 
PhnomPenh            1
Visite partagée      1
Sambor Preikuk       1
Cambodge             1
Bangkok              1
Name: count, Length: 93, dtype: int64

In [275]:
df_test = df_test[df_test['ville_finale_corrigee'] != 'Cambodge']


In [276]:
# Afficher seulement les lignes sans ville détectée
lignes_sans_ville = df_test[df_test['ville_detectee'].isna()]['nom']
print(lignes_sans_ville)


Series([], Name: nom, dtype: object)


In [277]:
# Corriger PhnomPenh -> Phnom Penh
df_test['ville_finale_corrigee'] = df_test['ville_finale_corrigee'].replace({'PhnomPenh': 'Phnom Penh'})

# Supprimer la ligne qui contient "Visite partagée"
df_test = df_test[df_test['ville_finale_corrigee'] != 'Visite partagée']

# Vérifier que tout est bien propre
print(df_test['ville_finale_corrigee'].value_counts())


ville_finale_corrigee
Siem Reap                    435
Phnom Penh                   135
Kampot                        25
Battambang                    22
Angkor Wat                    17
                            ... 
Explorer Phnom Penh            1
Cascade de Ta Tai              1
Visite privée de 13 jours      1
Sambor Preikuk                 1
Bangkok                        1
Name: count, Length: 90, dtype: int64


In [279]:
# Corriger "Explorer Phnom Penh" en "Phnom Penh"
df_test['ville_finale_corrigee'] = df_test['ville_finale_corrigee'].replace({'Explorer Phnom Penh': 'Phnom Penh'})

# Supprimer les lignes avec des pseudo-villes (mettre entre [ ] pour que ce soit une liste !)
df_test = df_test[~df_test['ville_finale_corrigee'].isin(['Visite privée de 13 jours'])]

# Vérifier que tout est propre
print(df_test['ville_finale_corrigee'].value_counts())



ville_finale_corrigee
Siem Reap                         435
Phnom Penh                        136
Kampot                             25
Battambang                         22
Angkor Wat                         17
                                 ... 
Excursion privée de deux jours      1
Koh Rong Samloem                    1
Cascade de Ta Tai                   1
Sambor Preikuk                      1
Bangkok                             1
Name: count, Length: 88, dtype: int64


In [280]:
# Supprimer les lignes avec des pseudo-villes (mettre entre [ ] pour que ce soit une liste !)
df_test = df_test[~df_test['ville_finale_corrigee'].isin(['Excursion privée de deux jours'])]

In [281]:
print(df_test['ville_finale_corrigee'].value_counts())

ville_finale_corrigee
Siem Reap            435
Phnom Penh           136
Kampot                25
Battambang            22
Angkor Wat            17
                    ... 
Angkor Sunrise         1
Koh Rong Samloem       1
Cascade de Ta Tai      1
Sambor Preikuk         1
Bangkok                1
Name: count, Length: 87, dtype: int64


In [282]:
print(df_test['ville_finale_corrigee'].unique())


['Siem Reap' 'HCM' 'Phnom Penh' 'Battambang' 'Montagne Kulen'
 'Ho Chi Minh Ville' 'Kampot' 'Ho Chi Minh' 'HCM City' 'Preah Vihear'
 'Beng Mealea' 'Phnom Kulen' 'Explorer Sihanoukville' 'Campagne de Kampot'
 'Visite privée de Kampot' 'Kick-Boxing'
 "Ferme de la soie d'Angkor et Ferme de la soie du Lotus"
 'Excursion dîner au coucher du soleil' 'Angkor Wat' 'Silk Island'
 'Sihanoukville' 'Croisière au coucher du soleil' 'Kampong Phluk' 'Phare'
 'Phnom Tamao' 'Bokor' 'Île de la Soie'
 'Visite de la ferme de poivre de Kampot' "Visite de l'après-midi"
 'Krong Siem Reap' 'Delta du Mékong 2 jours'
 "Excursion d'une demi-journée" 'Koh Ker' 'Parc de Phnom Kulen'
 'Village flottant de Kompong Khleang' 'De Siem Reap' 'Tuk tuk du matin'
 "Région d'Angkor" 'Les champs de la mort de Phnom Penh' 'Tonle Sap'
 'Excursion à Sihanoukville' 'Visite en petit groupe'
 "Sanctuaire d'éléphants du Cambodge" 'Hô Chi Minh-Ville' 'Vietnam' 'Kep'
 'Kompong Phluk et le lac Tonlé Sap' 'Banteay Srei' 'Visite privée'

In [283]:
# Nombre de valeurs uniques dans ta colonne de villes
print("Nombre de villes/lieux détectés :", df_test['ville_finale_corrigee'].nunique())

# Liste des villes/lieux uniques pour les examiner
print(df_test['ville_finale_corrigee'].unique())


Nombre de villes/lieux détectés : 87
['Siem Reap' 'HCM' 'Phnom Penh' 'Battambang' 'Montagne Kulen'
 'Ho Chi Minh Ville' 'Kampot' 'Ho Chi Minh' 'HCM City' 'Preah Vihear'
 'Beng Mealea' 'Phnom Kulen' 'Explorer Sihanoukville' 'Campagne de Kampot'
 'Visite privée de Kampot' 'Kick-Boxing'
 "Ferme de la soie d'Angkor et Ferme de la soie du Lotus"
 'Excursion dîner au coucher du soleil' 'Angkor Wat' 'Silk Island'
 'Sihanoukville' 'Croisière au coucher du soleil' 'Kampong Phluk' 'Phare'
 'Phnom Tamao' 'Bokor' 'Île de la Soie'
 'Visite de la ferme de poivre de Kampot' "Visite de l'après-midi"
 'Krong Siem Reap' 'Delta du Mékong 2 jours'
 "Excursion d'une demi-journée" 'Koh Ker' 'Parc de Phnom Kulen'
 'Village flottant de Kompong Khleang' 'De Siem Reap' 'Tuk tuk du matin'
 "Région d'Angkor" 'Les champs de la mort de Phnom Penh' 'Tonle Sap'
 'Excursion à Sihanoukville' 'Visite en petit groupe'
 "Sanctuaire d'éléphants du Cambodge" 'Hô Chi Minh-Ville' 'Vietnam' 'Kep'
 'Kompong Phluk et le lac Tonl

In [284]:
df_test['ville_finale_corrigee'] = df_test['ville_finale_corrigee'].replace({
    'HCM': 'Ho Chi Minh',
    'HCM City': 'Ho Chi Minh',
    'Ho Chi Minh Ville': 'Ho Chi Minh',
    'Hô Chi Minh-Ville': 'Ho Chi Minh',
    'Krong Siem Reap': 'Siem Reap',
    'Explorer Sihanoukville': 'Sihanoukville',
    'Campagne de Kampot': 'Kampot',
    'Visite privée de Kampot': 'Kampot',
    'Phnom Phnom': 'Phnom Penh',
    'Explorer Phnom Penh': 'Phnom Penh',
})


In [286]:
import pandas as pd

# Liste des villes et sites touristiques au Cambodge à garder
lieux_cambodge = [
    'Siem Reap', 'Phnom Penh', 'Battambang', 'Kampot', 'Kratie', 'Kompong Thom',
    'Kampong Cham', 'Preah Vihear', 'Sihanoukville', 'Kep', 'Koh Rong Samloem',
    'Angkor Wat', 'Banteay Srei', 'Koh Ker', 'Phnom Bakheng', 'Montagne Kulen',
    'Parc de Phnom Kulen', 'Phnom Da', 'Phnom Tamao', 'Village flottant de Kompong Khleang',
    'Kompong Phluk', 'Kompong Phluk et le lac Tonlé Sap', 'Silk Island', 'Banann',
    'Cai Rang', 'Tonle Sap', 'Delta du Mékong', 'Sambor Preikuk', 'Bokor'
]

# Corriger les doublons ou fautes de frappe si nécessaire
df_test['ville_finale_corrigee'] = df_test['ville_finale_corrigee'].replace({
    'PhnomPenh': 'Phnom Penh',
    'Explorer Phnom Penh': 'Phnom Penh'
})

# Filtrer : ne garder que les lieux présents dans la liste
df_test = df_test[df_test['ville_finale_corrigee'].isin(lieux_cambodge)]

# Vérifier le résultat
print("Nombre de lieux restants :", len(df_test))
print(df_test['ville_finale_corrigee'].value_counts())


Nombre de lieux restants : 703
ville_finale_corrigee
Siem Reap                              437
Phnom Penh                             137
Kampot                                  27
Battambang                              22
Angkor Wat                              17
Koh Ker                                 10
Sihanoukville                            8
Montagne Kulen                           7
Preah Vihear                             5
Tonle Sap                                5
Banteay Srei                             4
Bokor                                    3
Kep                                      3
Kratie                                   2
Delta du Mékong                          2
Silk Island                              1
Phnom Tamao                              1
Parc de Phnom Kulen                      1
Cai Rang                                 1
Kompong Phluk et le lac Tonlé Sap        1
Village flottant de Kompong Khleang      1
Phnom Bakheng                            1
P

In [287]:
# Nombre de valeurs uniques dans ta colonne de villes
print("Nombre de villes/lieux détectés :", df_test['ville_finale_corrigee'].nunique())

# Liste des villes/lieux uniques pour les examiner
print(df_test['ville_finale_corrigee'].unique())

Nombre de villes/lieux détectés : 29
['Siem Reap' 'Phnom Penh' 'Battambang' 'Montagne Kulen' 'Kampot'
 'Preah Vihear' 'Sihanoukville' 'Angkor Wat' 'Silk Island' 'Phnom Tamao'
 'Bokor' 'Koh Ker' 'Parc de Phnom Kulen'
 'Village flottant de Kompong Khleang' 'Tonle Sap' 'Kep'
 'Kompong Phluk et le lac Tonlé Sap' 'Banteay Srei' 'Cai Rang' 'Phnom Da'
 'Phnom Bakheng' 'Delta du Mékong' 'Kratie' 'Kompong Thom' 'Kompong Phluk'
 'Banann' 'Kampong Cham' 'Koh Rong Samloem' 'Sambor Preikuk']


In [288]:
df_test['nom'] = df_test['nom'].str.strip()  # enlève les espaces en début/fin
df_test['nom'] = df_test['nom'].str.replace('\s+', ' ', regex=True)  # réduit les multiples espaces


In [290]:
print(df_test['nom'].value_counts().head(20))


nom
Siem Reap : Angkor Wat : visite en petit groupe au lever ou au coucher du soleil             1
Au départ de Siem Reap : Tour en bateau du village flottant de Kampong Phluk                 1
Siem Reap : Visite de 2 jours à Angkor Wat avec lever et coucher de soleil                   1
Phnom Penh : Les champs de la mort et le musée du génocide de Tuol Sleng                     1
Battambang : Tuk-Tuk, train de bambou, grottes de Killing/Bat, coucher de soleil             1
Siem Reap : 1 jour de visite d'Angkor avec guide francophone                                 1
Phnom Penh : Croisière au coucher du soleil avec bières illimitées et buffet barbecue        1
Montagne Kulen : Visite en petit groupe et déjeuner pique-nique                              1
Siem Reap : Kampong Phluk Floating Village Tour en bateau                                    1
Siem Reap : excursion à la montagne Kulen, à Beng Mealea et au Tonlé Sap                     1
Kampot : Visite d'une ferme de poivre, d'un ch

In [292]:
remplacements_nom = {
    "Excursion d'une journée complète": "Excursion journée",
    "Visite privée de": "Visite privée",
    "Excursion à la montagne Kulen, à Beng Mealea et au Tonlé Sap": "Excursion Angkor & Tonlé Sap",
    "1 jour de visite d'Angkor avec guide francophone": "Visite Angkor 1 jour",
    # tu peux ajouter d'autres patterns
}

for k, v in remplacements_nom.items():
    df_test['nom'] = df_test['nom'].str.replace(k, v, regex=False)


In [294]:
df_test['nom'] = df_test['nom'].str.replace("'", "''")


In [297]:
df_attractions['nom'] = df_test['nom']
df_attractions['ville'] = df_test['ville_finale_corrigee']


In [299]:
df_attractions.head(30)

,nom,ville,duree,note,avis,prix
0,Siem Reap : Angkor Wat : visite en petit group...,Siem Reap,9.0,4.9,6779,13.0
1,Au départ de Siem Reap : Tour en bateau du vil...,Siem Reap,6.0,4.9,4952,17.0
2,Siem Reap : Visite de 2 jours à Angkor Wat ave...,Siem Reap,48.0,4.9,3481,25.0
3,NaN,NaN,48.0,4.5,1299,73.0
4,Phnom Penh : Les champs de la mort et le musée...,Phnom Penh,4.0,4.8,2613,16.0
5,"Battambang : Tuk-Tuk, train de bambou, grottes...",Battambang,12.0,4.9,1030,8.0
6,Siem Reap : Visite Angkor 1 jour,Siem Reap,8.0,4.9,147,43.0
7,Phnom Penh : Croisière au coucher du soleil av...,Phnom Penh,1.5,4.5,817,8.0
8,Montagne Kulen : Visite en petit groupe et déj...,Montagne Kulen,8.0,4.9,1177,41.0
9,Siem Reap : Kampong Phluk Floating Village Tou...,Siem Reap,6.0,4.9,3013,16.0


In [300]:
# Supprimer les lignes où 'nom' ou 'ville' sont NaN ou vides
df_attractions = df_attractions.dropna(subset=['nom', 'ville'])

In [303]:
#print(df_attractions.shape)
df_attractions.head()

,nom,ville,duree,note,avis,prix
0,Siem Reap : Angkor Wat : visite en petit group...,Siem Reap,9.0,4.9,6779,13.0
1,Au départ de Siem Reap : Tour en bateau du vil...,Siem Reap,6.0,4.9,4952,17.0
2,Siem Reap : Visite de 2 jours à Angkor Wat ave...,Siem Reap,48.0,4.9,3481,25.0
4,Phnom Penh : Les champs de la mort et le musée...,Phnom Penh,4.0,4.8,2613,16.0
5,"Battambang : Tuk-Tuk, train de bambou, grottes...",Battambang,12.0,4.9,1030,8.0


In [306]:
# Garder pour df_attractions
df_attractions.to_csv("attractions.csv", index=False, encoding="utf-8")

In [309]:

# Charger les fichiers
df_hotels = pd.read_csv("hotels.csv")
df_attractions = pd.read_csv("attractions.csv")

# 1. Extraire toutes les villes uniques
villes_uniques = pd.concat([df_hotels["ville"], df_attractions["ville"]]).dropna().unique()
df_villes = pd.DataFrame({"ville": sorted(villes_uniques)})

# 2. Créer un ID pour chaque ville
df_villes["id"] = range(1, len(df_villes)+1)

# 3. Remplacer les noms de villes par leurs ID
# Pour les hôtels
df_hotels = df_hotels.merge(df_villes, left_on="ville", right_on="ville", how="left")
df_hotels.drop(columns=["ville","ville"], inplace=True)
df_hotels.rename(columns={"id": "ville_id"}, inplace=True)

# Pour les attractions
df_attractions = df_attractions.merge(df_villes, left_on="ville", right_on="ville", how="left")
df_attractions.drop(columns=["ville"], inplace=True)
df_attractions.rename(columns={"id": "ville_id"}, inplace=True)

# 4. Sauvegarder les fichiers finaux
df_villes.to_csv("villes_final.csv", index=False)
df_hotels.to_csv("hotels_final.csv", index=False)
df_attractions.to_csv("attractions_final.csv", index=False)
